In [62]:
import itertools
import pandas as pd
import statistics
    
def calculateKendallTauDistance(listA, listB):
    distance = 0
    combinations = []
    for a in listA:
        for b in listB:
            if a != b:
                combinations.append(sorted([a, b]))
    pairs = list(set(map(tuple, combinations)))
    for x, y in pairs:
        a = listA.index(x) - listA.index(y)
        b = listB.index(x) - listB.index(y)
        if a * b < 0:
            distance += 1
    return distance

def partialScoring(dropOutOrder, playerPredictionsArray):
    placeHolders1 = list(map(chr, range(97, 97 + len(playerPredictionsArray) - 1)))
    placeHolders2 = list(map(chr, range(97, 97 + len(playerPredictionsArray) - 1)))
    
    predictionsWithPlaceHolders = []
    dropsWithPlaceHolders = []
    
    for candidate in playerPredictionsArray:
        if candidate not in dropOutOrder: # candidate hasn't dropped yet
            predictionsWithPlaceHolders.append(placeHolders1.pop())
        else: # candidate has dropped
            predictionsWithPlaceHolders.append(candidate)
                
    for i, candidate in enumerate(playerPredictionsArray):
        if i < len(dropOutOrder):
            dropsWithPlaceHolders.append(dropOutOrder[i])
        else:
            dropsWithPlaceHolders.append(placeHolders2.pop())
        
    print(predictionsWithPlaceHolders)
    print(dropsWithPlaceHolders)
    
    return calculateKendallTauDistance(dropsWithPlaceHolders, predictionsWithPlaceHolders);

df = pd.read_csv("../2024/leaderboard/data/submissions2024.csv")
df = df[['leaderboardAlias', 'prediction']]

In [64]:
display(df.tail(1))

,leaderboardAlias,prediction
49,Wisdom of the crowd,"hurd,elder,suarez,hutchinson,burgum,christie,s..."


In [119]:
averageOrder = ['hurd','elder','suarez','hutchinson','burgum','christie','scott','haley','ramaswamy','pence','desantis','trump']
reversed_list = averageOrder[::-1]
dropOutOrder = ['desantis', 'hurd', 'elder']

partialScoring(reversed_list, averageOrder)

['hurd', 'elder', 'suarez', 'hutchinson', 'burgum', 'christie', 'scott', 'haley', 'ramaswamy', 'pence', 'desantis', 'trump']
['trump', 'desantis', 'pence', 'ramaswamy', 'haley', 'scott', 'christie', 'burgum', 'hutchinson', 'suarez', 'elder', 'hurd']


66